In [2]:
import os
import re
import math
import cv2
import pandas as pd
import numpy as np

from tqdm import tqdm
from glob import glob
from tools.settings import *
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tools.train_val_test_spliter import split

### Pre Processing

In [35]:
# To split dataset if already splits folder already exits no need to run it
split()

Splitting the ginen dataset into Train Test=0.2 Validation=0.2
Done


In [36]:
train = pd. read_csv(os.path.join(dataset_path, "train.csv"))
test = pd. read_csv(os.path.join(dataset_path, "test.csv"))
val = pd. read_csv(os.path.join(dataset_path, "val.csv"))

In [37]:
train.head()

,Video_url,action
0,H:/THESIS/dataset\kick/kick_192.mp4,kick
1,H:/THESIS/dataset\kick/kick_197.mp4,kick
2,H:/THESIS/dataset\slap/Slap_160.mp4,slap
3,H:/THESIS/dataset\kick/kick_183.mp4,kick
4,H:/THESIS/dataset\kick/kick_17.mp4,kick


In [5]:
test.head()

,Video_url,action
0,H:/THESIS/dataset\kick/kick_191.mp4,kick
1,H:/THESIS/dataset\slap/Slap_158.mp4,slap
2,H:/THESIS/dataset\kick/kick_162.mp4,kick
3,H:/THESIS/dataset\punch/punch_128.mp4,punch
4,H:/THESIS/dataset\punch/punch_187.mp4,punch


In [6]:
val.head()

,Video_url,action
0,H:/THESIS/dataset\slap/slap_136.mp4,slap
1,H:/THESIS/dataset\kick/kick_187.mp4,kick
2,H:/THESIS/dataset\punch/punch_172.mp4,punch
3,H:/THESIS/dataset\punch/punch_111.mp4,punch
4,H:/THESIS/dataset\slap/slap_26.mp4,slap


In [5]:
print(train.shape)
print(test.shape)
print(val.shape)

(370, 2)
(123, 2)
(123, 2)


In [38]:
def convert_to_frame(data, folder_name):
    '''
    Generated filenames format dataset_path/folder_name/video_name_frame{number}_action.jpg
    '''
    directory = os.path.join(dataset_path, folder_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    for i in tqdm(range(data.shape[0])):
        video_file = data['Video_url'][i]
        action = data['action'][i]
        video_name_list = video_file.split('/')[-1].split('.')
        video_name_list = video_name_list[:-1]
        video_name = ""
        for n in video_name_list:
            video_name += n
        # capturing the video from the given path
        capture = cv2.VideoCapture(video_file) 
        #frame rate
        frame_rate = capture.get(5)
        count = 0
        while(capture.isOpened()):
            #current frame number
            frame_id = capture.get(1) 
            read_correctly, frame = capture.read()
            if not read_correctly:
                break
            if (frame_id % math.floor(frame_rate) == 0):
                # storing the frames in a new folder named train_1
                filename = directory + "/" + video_name + "_frame{}_".format(count) + action +".jpg"
                count += 1
                cv2.imwrite(filename, frame)
        capture.release()
    print("Successfully Converted")

In [39]:
convert_to_frame(train, train_frames_path_name)

100%|████████████████████████████████████████████████████████████████████████████████| 370/370 [03:47<00:00,  1.63it/s]

Successfully Converted


In [40]:
convert_to_frame(val, val_frames_path_name)

100%|████████████████████████████████████████████████████████████████████████████████| 123/123 [01:10<00:00,  1.75it/s]

Successfully Converted


In [41]:
def create_paths_csv(directory, file_name):
    images = os.listdir(directory)
    images_path_list = []
    images_action_list = [] 
    for image in images:
        images_path_list.append(directory + image)
        images_action_list.append(image.split('.')[0].split('_')[-1])
    df = pd.DataFrame()
    df['image'] = images_path_list
    df['action'] = images_action_list
    print(os.path.join(dataset_path, file_name+'.csv'))
    df.to_csv(os.path.join(dataset_path, file_name+'.csv'), index=False)

In [42]:
create_paths_csv(train_frames_path, train_frames_path_name)

H:/THESIS/dataset\train_frames.csv


In [43]:
create_paths_csv(val_frames_path, val_frames_path_name)

H:/THESIS/dataset\val_frames.csv


In [95]:
train_image = pd.read_csv(os.path.join(dataset_path, 'train_frames.csv'))
train_image.head()

,image,action
0,H:/THESIS/dataset\train_frames\kick_01_frame0_...,kick
1,H:/THESIS/dataset\train_frames\kick_01_frame1_...,kick
2,H:/THESIS/dataset\train_frames\kick_01_frame2_...,kick
3,H:/THESIS/dataset\train_frames\kick_01_frame3_...,kick
4,H:/THESIS/dataset\train_frames\kick_02_frame0_...,kick


In [96]:
print(train_image.shape)

(2171, 2)


In [97]:
val_image = pd.read_csv(os.path.join(dataset_path, 'val_frames.csv'))
val_image.head()

,image,action
0,H:/THESIS/dataset\val_frames\kick_02_frame0_ki...,kick
1,H:/THESIS/dataset\val_frames\kick_02_frame1_ki...,kick
2,H:/THESIS/dataset\val_frames\kick_07_frame0_ki...,kick
3,H:/THESIS/dataset\val_frames\kick_07_frame1_ki...,kick
4,H:/THESIS/dataset\val_frames\kick_07_frame2_ki...,kick


In [98]:
print(val_image.shape)

(921, 2)


In [99]:
action_values = list(train_image['action'].unique())
action_values

['kick', 'punch', 'slap']

In [100]:
def create_class_columns(df):
    for value in action_values:
        df[value] = np.where(df['action'].str.contains(value), 1, 0)
    df.drop('action', axis='columns', inplace=True)

In [101]:
create_class_columns(train_image)
train_image.head()

,image,kick,punch,slap
0,H:/THESIS/dataset\train_frames\kick_01_frame0_...,1,0,0
1,H:/THESIS/dataset\train_frames\kick_01_frame1_...,1,0,0
2,H:/THESIS/dataset\train_frames\kick_01_frame2_...,1,0,0
3,H:/THESIS/dataset\train_frames\kick_01_frame3_...,1,0,0
4,H:/THESIS/dataset\train_frames\kick_02_frame0_...,1,0,0


In [102]:
create_class_columns(val_image)
val_image.head()

,image,kick,punch,slap
0,H:/THESIS/dataset\val_frames\kick_02_frame0_ki...,1,0,0
1,H:/THESIS/dataset\val_frames\kick_02_frame1_ki...,1,0,0
2,H:/THESIS/dataset\val_frames\kick_07_frame0_ki...,1,0,0
3,H:/THESIS/dataset\val_frames\kick_07_frame1_ki...,1,0,0
4,H:/THESIS/dataset\val_frames\kick_07_frame2_ki...,1,0,0


In [103]:
def convert_to_array_and_split(image_data):
    image_value = []
    for i in tqdm(range(image_data.shape[0])):
        img = image.load_img(image_data['image'][i], target_size=(224,224,3))
        img = image.img_to_array(img)
        # normalizing the pixel value
        img = img / 255
        image_value.append(img)

    X = np.array(image_value)
    y = image_data
    y.drop('image', axis='columns', inplace=True)
    return X, y

In [104]:
X_train, y_train = convert_to_array_and_split(train_image)
print(X_train.shape)

100%|██████████████████████████████████████████████████████████████████████████████| 2171/2171 [00:50<00:00, 43.01it/s]


(2171, 224, 224, 3)


In [105]:
X_val, y_val = convert_to_array_and_split(val_image)
print(X_val.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 921/921 [00:18<00:00, 49.55it/s]


(921, 224, 224, 3)


In [106]:
y_train.head()

,kick,punch,slap
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [107]:
y_val.head()

,kick,punch,slap
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


### Processing With Pre-trained VGG16 with imagenet weight

In [55]:
from keras.applications.vgg16 import VGG16
'''This model was trained on a dataset that has 1,000 classes. 
include_top = False will remove the last layer of this model so that we can tune it as per our need.
'''
base_model = VGG16(weights='imagenet', include_top=False)

In [56]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(2171, 7, 7, 512)

In [57]:
X_val = base_model.predict(X_val)
X_val.shape

(921, 7, 7, 512)

In [58]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(2171, 7*7*512)
X_val = X_val.reshape(921, 7*7*512)

In [59]:
# normalizing the pixel values
max_pixel = X_train.max()
X_train = X_train / max_pixel
X_val = X_val / max_pixel
print(X_train.shape)
print(X_val.shape)

(2171, 25088)
(921, 25088)


### Deep Neural Network model with 5 layers and3 output neurons

In [118]:
# The input shape will be 25,088
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 1024)              25691136  
_________________________________________________________________
dropout_16 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_17 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 128)              

In [61]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_weight = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [62]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[mcp_weight], batch_size=128)

Epoch 1/50
17/17 [==============================] - 5s 319ms/step - loss: 1.1953 - accuracy: 0.3482 - val_loss: 1.0849 - val_accuracy: 0.3844
Epoch 2/50
17/17 [==============================] - 7s 414ms/step - loss: 1.1157 - accuracy: 0.3676 - val_loss: 1.0657 - val_accuracy: 0.5613
Epoch 3/50
17/17 [==============================] - 6s 377ms/step - loss: 1.0584 - accuracy: 0.4067 - val_loss: 0.9697 - val_accuracy: 0.5603
Epoch 4/50
17/17 [==============================] - 7s 439ms/step - loss: 0.9405 - accuracy: 0.5108 - val_loss: 0.8221 - val_accuracy: 0.6211
Epoch 5/50
17/17 [==============================] - 7s 411ms/step - loss: 0.8461 - accuracy: 0.5758 - val_loss: 0.7219 - val_accuracy: 0.6710
Epoch 6/50
17/17 [==============================] - 7s 395ms/step - loss: 0.7013 - accuracy: 0.6642 - val_loss: 0.5837 - val_accuracy: 0.7557
Epoch 7/50
17/17 [==============================] - 7s 430ms/step - loss: 0.5944 - accuracy: 0.7462 - val_loss: 0.5284 - val_accuracy: 0.7459
Epoch 

### Applying Built model  on Test data and calculate accuracy with VGG16 

In [119]:
model.load_weights("weight.hdf5")
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 1024)              25691136  
_________________________________________________________________
dropout_16 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_17 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 128)              

In [120]:
from scipy import stats as s
predict = []
actual = []
if not os.path.exists(test_frames_path):
    os.makedirs(test_frames_path)

for i in tqdm(range(test.shape[0])):
    video_file = test['Video_url'][i]
    action = test['action'][i]
    video_name_list = video_file.split('/')[-1].split('.')
    video_name_list = video_name_list[:-1]
    video_name = ""
    for n in video_name_list:
        video_name += n
    # capturing the video from the given path
    capture = cv2.VideoCapture(video_file) 
    #frame rate
    frame_rate = capture.get(5)
    count = 0
    files = glob(test_frames_path + '/*')
    #removing all files from folder
    for f in files:
        os.remove(f)
    while(capture.isOpened()):
        #current frame number
        frame_id = capture.get(1) 
        read_correctly, frame = capture.read()
        if not read_correctly:
            break
        if (frame_id % math.floor(frame_rate) == 0):
            # storing the frames in a new folder named train_1
            filename = test_frames_path + "/" + video_name + "_frame{}_".format(count) + action +".jpg"
            count += 1
            cv2.imwrite(filename, frame)
    capture.release()
    
    # reading all the frames from temp folder
    images = glob(test_frames_path + '/*.jpg')
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img / 255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    # predicting tags for each array
    prediction = np.argmax(model.predict(prediction_images), axis=-1)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y_train.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(action)

100%|████████████████████████████████████████████████████████████████████████████████| 123/123 [02:32<00:00,  1.24s/it]


In [89]:
y_train.columns.values[0]

'kick'

In [121]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

92.6829268292683

### Processing With Pre-trained InceptionV3 with imagenet weight

In [80]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model = InceptionV3(include_top = False, weights = 'imagenet')

87916544/87910968 [==============================] - 43s 0us/step


In [81]:
# extracting features for training frames
X_train = inception_model.predict(X_train)
X_train.shape

(2171, 5, 5, 2048)

In [82]:
X_val = inception_model.predict(X_val)
X_val.shape

(921, 5, 5, 2048)

In [83]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(2171, 5*5*2048)
X_val = X_val.reshape(921, 5*5*2048)

In [84]:
# normalizing the pixel values
max_pixel = X_train.max()
X_train = X_train / max_pixel
X_val = X_val / max_pixel
print(X_train.shape)
print(X_val.shape)

(2171, 51200)
(921, 51200)


In [85]:
# The input shape will be 51200
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(51200,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 1024)              52429824  
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)              

In [86]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_weight = ModelCheckpoint('weight1.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [88]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[mcp_weight], batch_size=128)

Epoch 1/50
17/17 [==============================] - 8s 455ms/step - loss: 0.0459 - accuracy: 0.9848 - val_loss: 0.6965 - val_accuracy: 0.9066
Epoch 2/50
17/17 [==============================] - 8s 469ms/step - loss: 0.0666 - accuracy: 0.9797 - val_loss: 0.6176 - val_accuracy: 0.8990
Epoch 3/50
17/17 [==============================] - 8s 458ms/step - loss: 0.0527 - accuracy: 0.9843 - val_loss: 0.6065 - val_accuracy: 0.8979
Epoch 4/50
17/17 [==============================] - 8s 452ms/step - loss: 0.0616 - accuracy: 0.9816 - val_loss: 0.6853 - val_accuracy: 0.9034
Epoch 5/50
17/17 [==============================] - 7s 441ms/step - loss: 0.0488 - accuracy: 0.9857 - val_loss: 0.7090 - val_accuracy: 0.9034
Epoch 6/50
17/17 [==============================] - 7s 440ms/step - loss: 0.0443 - accuracy: 0.9834 - val_loss: 0.5595 - val_accuracy: 0.9034
Epoch 7/50
17/17 [==============================] - 7s 397ms/step - loss: 0.0320 - accuracy: 0.9889 - val_loss: 0.6417 - val_accuracy: 0.9066
Epoch 

### Applying Built model  on Test data and calculate accuracy with InceptionV3

In [92]:
from scipy import stats as s
predict = []
actual = []
if not os.path.exists(test_frames_path):
    os.makedirs(test_frames_path)

for i in tqdm(range(test.shape[0])):
    video_file = test['Video_url'][i]
    action = test['action'][i]
    video_name_list = video_file.split('/')[-1].split('.')
    video_name_list = video_name_list[:-1]
    video_name = ""
    for n in video_name_list:
        video_name += n
    # capturing the video from the given path
    capture = cv2.VideoCapture(video_file) 
    #frame rate
    frame_rate = capture.get(5)
    count = 0
    files = glob(test_frames_path + '/*')
    #removing all files from folder
    for f in files:
        os.remove(f)
    while(capture.isOpened()):
        #current frame number
        frame_id = capture.get(1) 
        read_correctly, frame = capture.read()
        if not read_correctly:
            break
        if (frame_id % math.floor(frame_rate) == 0):
            # storing the frames in a new folder named train_1
            filename = test_frames_path + "/" + video_name + "_frame{}_".format(count) + action +".jpg"
            count += 1
            cv2.imwrite(filename, frame)
    capture.release()
    
    # reading all the frames from temp folder
    images = glob(test_frames_path + '/*.jpg')
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img / 255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = inception_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 5*5*2048)
    # predicting tags for each array
    prediction = np.argmax(model.predict(prediction_images), axis=-1)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y_train.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(action)

100%|████████████████████████████████████████████████████████████████████████████████| 123/123 [01:50<00:00,  1.12it/s]


In [93]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

91.869918699187

### Processing With Pre-trained ResNet50 with imagenet weight

In [94]:
from tensorflow.keras.applications import ResNet50
resnet_model = ResNet50(include_top=False, weights="imagenet")

94773248/94765736 [==============================] - 45s 0us/step


In [108]:
X_train = resnet_model.predict(X_train)
X_train.shape

(2171, 7, 7, 2048)

In [109]:
X_val = resnet_model.predict(X_val)
X_val.shape

(921, 7, 7, 2048)

In [110]:
X_train = X_train.reshape(2171, 7*7*2048)
X_val = X_val.reshape(921, 7*7*2048)

In [111]:
max_pixel = X_train.max()
X_train = X_train / max_pixel
X_val = X_val / max_pixel
print(X_train.shape)
print(X_val.shape)

(2171, 100352)
(921, 100352)


In [112]:
# The input shape will be 100352
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(100352,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 1024)              102761472 
_________________________________________________________________
dropout_12 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)              

In [113]:
from keras.callbacks import ModelCheckpoint
mcp_weight = ModelCheckpoint('weight2.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [114]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[mcp_weight], batch_size=128)

Epoch 1/50
17/17 [==============================] - 23s 1s/step - loss: 1.2903 - accuracy: 0.3459 - val_loss: 1.1094 - val_accuracy: 0.3181
Epoch 2/50
17/17 [==============================] - 27s 2s/step - loss: 1.1570 - accuracy: 0.3399 - val_loss: 1.0996 - val_accuracy: 0.3181
Epoch 3/50
17/17 [==============================] - 26s 2s/step - loss: 1.1123 - accuracy: 0.3496 - val_loss: 1.0926 - val_accuracy: 0.3865
Epoch 4/50
17/17 [==============================] - 14s 809ms/step - loss: 1.1080 - accuracy: 0.3542 - val_loss: 1.0932 - val_accuracy: 0.3865
Epoch 5/50
17/17 [==============================] - 14s 820ms/step - loss: 1.1026 - accuracy: 0.3671 - val_loss: 1.0942 - val_accuracy: 0.3865
Epoch 6/50
17/17 [==============================] - 14s 800ms/step - loss: 1.0995 - accuracy: 0.3630 - val_loss: 1.0952 - val_accuracy: 0.3865
Epoch 7/50
17/17 [==============================] - 14s 797ms/step - loss: 1.0960 - accuracy: 0.3657 - val_loss: 1.0947 - val_accuracy: 0.3865
Epoch 8/

### Applying Built model  on Test data and calculate accuracy with Resnet50

In [116]:
from scipy import stats as s
predict = []
actual = []
if not os.path.exists(test_frames_path):
    os.makedirs(test_frames_path)

for i in tqdm(range(test.shape[0])):
    video_file = test['Video_url'][i]
    action = test['action'][i]
    video_name_list = video_file.split('/')[-1].split('.')
    video_name_list = video_name_list[:-1]
    video_name = ""
    for n in video_name_list:
        video_name += n
    # capturing the video from the given path
    capture = cv2.VideoCapture(video_file) 
    #frame rate
    frame_rate = capture.get(5)
    count = 0
    files = glob(test_frames_path + '/*')
    #removing all files from folder
    for f in files:
        os.remove(f)
    while(capture.isOpened()):
        #current frame number
        frame_id = capture.get(1) 
        read_correctly, frame = capture.read()
        if not read_correctly:
            break
        if (frame_id % math.floor(frame_rate) == 0):
            # storing the frames in a new folder named train_1
            filename = test_frames_path + "/" + video_name + "_frame{}_".format(count) + action +".jpg"
            count += 1
            cv2.imwrite(filename, frame)
    capture.release()
    
    # reading all the frames from temp folder
    images = glob(test_frames_path + '/*.jpg')
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img / 255
        prediction_images.append(img)
        
    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = resnet_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*2048)
    # predicting tags for each array
    prediction = np.argmax(model.predict(prediction_images), axis=-1)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y_train.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(action)

100%|████████████████████████████████████████████████████████████████████████████████| 123/123 [02:15<00:00,  1.10s/it]


In [117]:
accuracy_score(predict, actual)*100

66.66666666666666